# Projet No 2 : Comparaison des techniques de vectorisation
Objectif:\
Appliquer différentes techniques de vectorisation sur un corpus thématique et comparer leurs
performances en termes de similarité entre les documents et une liste de mots clés.

Constitution du corpus thématique \
○ Utiliser des ensembles de données publics pour créer un corpus thématique.
Vous pendre des articles de presse en ligne comme l’AIB,

In [1]:
import requests
from bs4 import BeautifulSoup


# URL du site à scraper
url = 'https://www.aib.media/'

# Envoyer une requête à l'URL
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Failed to load page {url}")

# Parser le contenu HTML
soup = BeautifulSoup(response.content, 'html.parser')

# Extraire les articles
articles = []
for link in soup.find_all('a', class_='td-image-wrap'):  # Adapter le sélecteur selon la structure du site
    try:
        if link.get('href').startswith('http://') or link.get('href').startswith('https://'):
            # Envoyer une requête à l'URL
            response = requests.get(link.get('href'))
            if response.status_code == 200:
                # Parser le contenu HTML
                soup2 = BeautifulSoup(response.content, 'html.parser')

                for article in soup2.find_all('article'):  # Adapter le sélecteur selon la structure du site
                    try:
                        title = article.find('h1').get_text()
                        content = article.find('div', class_='td-post-content tagdiv-type').get_text()
                        articles.append({'title': title, 'content': content})
                    except AttributeError as e:
                            print("AttributeError")
                        
    except TypeError as e:
        if 'NoneType' in str(e) and 'not callable' in str(e):
            print("TypeError")
        else:
            raise  # Re-raise if it's a different TypeError



# Afficher les articles extraits
for article in articles:
    print(f"Title: {article['title']}\nContent: {article['content']}\n")

AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
AttributeError
Title: Il y a 80 ans, les Anglo-américains débarquent en Normandie, à la demande de la Russie
Content: Monde- 2e Guerre mondiale-CommémorationIl y a 80 ans, les Anglo-américains débarquent en Normandie, à la demande de la RussieOuagadougou, 06 juin 2024 (AIB) – Les Etats-Unis d’Amérique et la Grande Bretagne, les rares pays occidentaux encore libres face aux nazis, démarquent le 6 juin 1944 en Normandie, en France, pour ouvrir un front ouest contre l’Allemagne, à la demande de Staline.Le 80e anniversaire du débarquement de Normandie est aujourd’hui célébré avec faste par la France, en présence du président américain, Joe Biden et ukrainien, Volodymir Zelensky et d’une vingtaine de chef d’Etat. La Russie n’est pas conviée, au contraire de l’Ukraine.«F

Prétraitement des données :\
○ Nettoyer le texte en supprimant les stopwords, la ponctuation et en
appliquant des techniques de lemmatisation.

In [2]:
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# nltk.download() A exécuter pour la première fois
def nettoyer_texte(texte):
    # Conversion en minuscules
    texte = texte.lower()
    
    # Suppression de la ponctuation
    texte = texte.translate(str.maketrans('', '', string.punctuation))
    
    # Tokenisation
    tokens = word_tokenize(texte)
    
    # Suppression des mots vides
    stop_words = set(stopwords.words('french'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Suppression des chiffres
    tokens = [word for word in tokens if not word.isdigit()]
    
    # Suppression des espaces blancs
    tokens = [word.strip() for word in tokens if word.strip()]
    
    return tokens

# Exemple d'utilisation
# texte = "Ceci est un exemple de texte à nettoyer pour la modélisation de sujets."
# tokens_nettoyes = nettoyer_texte(texte)
# print(tokens_nettoyes)

for article in articles:
    article['title'] = nettoyer_texte(article['title'])
    article['content'] = nettoyer_texte(article['content'])


# Afficher les articles extraits
for article in articles:
    print(f"Title: {article['title']}\nContent: {article['content']}\n")
    


Title: ['a', 'an', 'angloaméricains', 'débarquent', 'normandie', 'demande', 'russie']
Content: ['monde', '2e', 'guerre', 'mondialecommémorationil', 'a', 'an', 'angloaméricains', 'débarquent', 'normandie', 'demande', 'russieouagadougou', 'juin', 'aib', '–', 'etatsunis', '’', 'amérique', 'grande', 'bretagne', 'rares', 'pay', 'occidentaux', 'encore', 'libres', 'face', 'nazi', 'démarquent', 'juin', 'normandie', 'france', 'ouvrir', 'front', 'ouest', 'contre', '’', 'allemagne', 'demande', 'stalinele', '80e', 'anniversaire', 'débarquement', 'normandie', 'aujourd', '’', 'hui', 'célébré', 'faste', 'france', 'présence', 'président', 'américain', 'joe', 'biden', 'ukrainien', 'volodymir', 'zelensky', '’', 'vingtaine', 'chef', '’', 'etat', 'russie', '’', 'conviée', 'contraire', '’', 'ukraine', '«', 'face', 'ceux', 'prétendent', 'changer', 'frontières', 'force', 'dignes', 'ceux', 'débarqué', 'ici', '»', 'a', 'déclaré', 'président', 'français', 'emmanuel', 'macron', '«', 'merci', 'peuple', 'ukrainien

Enregistrement des articles dans un dataframe

In [8]:
import pandas as pd
# Créer un DataFrame pandas à partir des articles
df = pd.DataFrame(articles)

# Enregistrer le DataFrame dans un fichier CSV
df.to_csv("articles.csv", index=False)

Vectorisation avec TF-IDF :

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assurez-vous que la variable articles est correctement définie avec les titres et les contenus nettoyés

# Récupérer les titres nettoyés
titres_nettoyes = [" ".join(article['title']) for article in articles]

# Récupérer les contenus nettoyés
contenus_nettoyes = [" ".join(article['content']) for article in articles]

# Fusionner les titres et les contenus en une seule liste de textes
textes_nettoyes = titres_nettoyes + contenus_nettoyes

# Initialiser le vectoriseur TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

# Appliquer la vectorisation TF-IDF aux données textuelles nettoyées
tfidf_matrix = tfidf_vectorizer.fit_transform(textes_nettoyes)

# Afficher la forme de la matrice TF-IDF
print("Shape of TF-IDF matrix:", tfidf_matrix.shape)


Shape of TF-IDF matrix: (166, 2141)


3. Application de différentes techniques de modélisation de sujets :
○ Utiliser des techniques telles que :
 

* Latent Dirichlet Allocation (LDA).\
Rappel: LDA (Latent Dirichlet Allocation) : Algorithme probabiliste qui génère des
distributions de mots pour chaque sujet et des distributions de sujets pour chaque
document.

In [4]:
from sklearn.decomposition import LatentDirichletAllocation

# Initialiser le modèle LDA
lda_model = LatentDirichletAllocation(n_components=10, random_state=42)  # Nombre de sujets à découvrir

# Appliquer LDA sur la matrice TF-IDF
lda_matrix = lda_model.fit_transform(tfidf_matrix)

# Afficher les sujets découverts
feature_names = tfidf_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda_model.components_):
    print(f"Sujet {topic_idx+1}:")
    print([feature_names[i] for i in topic.argsort()[:-10 - 1:-1]])


Sujet 1:
['bepc', 'vidéo', 'canular', 'organisation', 'irrégularités', 'professeurs', 'cameron', 'britannique', 'david', 'diplomatie']
Sujet 2:
['image', 'terroristes', 'russie', 'tita06', 'matériel', 'armées', 'burkinabè', 'nigérienne', 'morts', 'laba']
Sujet 3:
['accra', 'ghana', 'électricité', 'coupures', 'manifestation', 'ouédraogo', 'guerre', 'interview', 'contre', 'grande']
Sujet 4:
['district', 'sanitaire', 'pouytenga', 'sankmoney', 'tonne', 'offre', 'matériel', 'gouvernement', 'permis', 'plus']
Sujet 5:
['apprenantes', 'coiffure', 'couture', 'kourittenga', 'coupures', 'promotion', 'sortie', 'bénin', 'électricité', 'étape']
Sujet 6:
['arrêtées', 'marier', 'comores', 'voulu', 'agence', 'femmes', 'avoir', 'information', 'deux', 'burkina']
Sujet 7:
['real', 'madrid', 'champion', 'ligue', 'ministre', 'couture', 'formation', 'coupe', 'tous', 'fois']
Sujet 8:
['etat', 'snc', 'chef', 'bwaba', 'culture', 'peulhs', 'fraternité', 'projet', 'équipements', 'célèbrent']
Sujet 9:
['dosso', 'p

* Non-negative Matrix Factorization (NMF)\
NMF (Non-negative Matrix Factorization) : Technique factorisation de matrice utilisée
pour identifier les thèmes en décomposant le texte en composants non négatifs.

In [5]:
from sklearn.decomposition import NMF

# Initialiser le modèle NMF
nmf_model = NMF(n_components=10, random_state=42)  # Nombre de sujets à découvrir

# Appliquer NMF sur la matrice TF-IDF
nmf_matrix = nmf_model.fit_transform(tfidf_matrix)

# Afficher les sujets découverts
feature_names = tfidf_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(nmf_model.components_):
    print(f"Sujet {topic_idx+1}:")
    print([feature_names[i] for i in topic.argsort()[:-10 - 1:-1]])


Sujet 1:
['terroristes', 'matériel', 'armées', 'nigérienne', '20h50mn', 'nombreux', 'récupèrent', 'actualisée', 'burkinabè', 'tuent']
Sujet 2:
['accident', 'laba', 'blessés', 'morts', 'village', 'nationale', 'tita06', 'victimes', 'route', 'fait']
Sujet 3:
['raffinerie', 'niger', 'dosso', 'prépare', 'construire', 'projet', 'nouvelle', 'région', 'démarrage', 'ministre']
Sujet 4:
['image', 'russie', 'chef', 'normandie', 'etat', 'étrangères', 'audience', 'ministre', 'affaire', 'angloaméricains']
Sujet 5:
['district', 'sanitaire', 'pouytenga', 'sankmoney', 'offre', 'santé', 'matériel', 'agent', 'excellence', 'fds']
Sujet 6:
['coupures', 'électricité', 'accra', 'ghana', 'manifestation', 'contre', 'courant', 'lira', 'npp', 'pay']
Sujet 7:
['publication', 'auf', 'doctorants', 'compétences', 'trentaine', 'scientifiques', 'renforce', 'scientifique', 'burkina', 'article']
Sujet 8:
['couture', 'coiffure', 'apprenantes', 'promotion', 'kourittenga', 'sortie', 'elles', 'fille', 'formation', 'élèves']

* BERTopic\
 Les transformers, comme BERT, peuvent être utilisés pour la modélisation de sujets en extrayant des représentations sémantiques des documents et en les regroupant en sujets.

In [ ]:
#pip install bertopic --user
##textes_nettoyes

In [6]:
from bertopic import BERTopic

# Initialiser BERTopic
topic_model = BERTopic(language="french")
# Fit BERTopic sur la matrice TF-IDF
#topics, _ = topic_model.fit_transform(tfidf_matrix)
topics, _ = topic_model.fit_transform(textes_nettoyes)
# Afficher les sujets découverts
topic_model.get_topics()


{-1: [('santé', 0.06360552400796884),
  ('terroristes', 0.058951413368028664),
  ('matériel', 0.05824165200123392),
  ('agent', 0.05824165200123392),
  ('district', 0.05377701275116479),
  ('force', 0.052267534466955236),
  ('sanitaire', 0.047715139826835375),
  ('juin', 0.04128717299321444),
  ('permis', 0.04033275956337359),
  ('coupures', 0.03852824754063193)],
 0: [('burkina', 0.02896501461485774),
  ('cette', 0.020546061046385504),
  ('formation', 0.018566418997313164),
  ('plus', 0.01823969454204075),
  ('couture', 0.01696699577103601),
  ('ministre', 0.01688714515254734),
  ('président', 0.016774805833664294),
  ('photo', 0.016437127714924647),
  ('guerre', 0.015946018853119286),
  ('information', 0.015861614243241704)],
 1: [('projet', 0.1002809695763532),
  ('niger', 0.07517286964998489),
  ('raffinerie', 0.07452021233698682),
  ('tonne', 0.05381692824779598),
  ('ioba', 0.048283756313206354),
  ('région', 0.04726881932140557),
  ('production', 0.04654802164481796),
  ('constr

4. Interprétation des résultats :\
○ Analyser les sujets extraits par chaque technique.\
○ Identifier les mots clés associés à chaque sujet et interpréter le thème général

## Interprétation des sujets extraits par le modèle LDA


* Sujet 1: Éducation et examens

    Mots-clés: bepc, vidéo, canular, organisation, irrégularités, professeurs, cameron, britannique, david, diplomatie\
    Interprétation: Ce sujet semble traiter des aspects de l'éducation, notamment des examens comme le BEPC, des problèmes potentiels liés à l'organisation des examens, et des questions de diplomatie impliquant le Cameroun et la Grande-Bretagne.

* Sujet 2: Sécurité et terrorisme

    Mots-clés: image, terroristes, russie, tita06, matériel, armées, burkinabè, nigérienne, morts, laba\
    Interprétation: Ce sujet concerne la sécurité et le terrorisme, avec des références à des images associées au terrorisme, aux armées burkinabè et nigérienne, ainsi qu'à des événements mortels.

* Sujet 3: Manifestations et électricité au Ghana

    Mots-clés: accra, ghana, électricité, coupures, manifestation, ouédraogo, guerre, interview, contre, grande\
    Interprétation: Ce sujet évoque des manifestations et des problèmes d'électricité au Ghana, avec des références aux coupures d'électricité, à des manifestations à Accra, et à des discussions sur la guerre et l'opposition.

* Sujet 4: Gestion sanitaire et gouvernementale

    Mots-clés: district, sanitaire, pouytenga, sankmoney, tonne, offre, matériel, gouvernement, permis, plus\
    Interprétation: Ce sujet aborde la gestion sanitaire et gouvernementale, avec des références à des districts sanitaires, des offres de matériel, et des aspects liés à la gestion gouvernementale et administrative.

* Sujet 5: Formation professionnelle et électricité

    Mots-clés: apprenantes, coiffure, couture, kourittenga, coupures, promotion, sortie, bénin, électricité, étape\
    Interprétation: Ce sujet semble traiter de la formation professionnelle, en mettant en avant des activités telles que la coiffure et la couture, ainsi que des problèmes liés à l'électricité et aux coupures d'électricité.

* Sujet 6: Mariage et agence matrimoniale

    Mots-clés: arrêtées, marier, comores, voulu, agence, femmes, avoir, information, deux, burkina\
    Interprétation: Ce sujet semble traiter du mariage et des agences matrimoniales, avec des références à des arrestations liées à des mariages, des discussions sur les mariages aux Comores, et des informations sur les agences matrimoniales.

* Sujet 7: Sports et événements culturels

    Mots-clés: real, madrid, champion, ligue, ministre, couture, formation, coupe, tous, fois\
    Interprétation: Ce sujet aborde des thèmes liés au sport, notamment le Real Madrid et la Ligue des champions, ainsi que des événements culturels comme la couture et la formation.

* Sujet 8: Culture et projets communautaires

    Mots-clés: état, snc, chef, bwaba, culture, peulhs, fraternité, projet, équipements, célèbrent\
    Interprétation: Ce sujet semble traiter de la culture et des projets communautaires, avec des références à la fraternité, aux équipements communautaires, et aux célébrations culturelles.

* Sujet 9: Projets de construction et accidents

    Mots-clés: dosso, prépare, construire, raffinerie, niger, nouvelle, région, accident, laba, village\
    Interprétation: Ce sujet semble aborder des projets de construction et des accidents, avec des références à la préparation de projets de construction, à des accidents, et à des discussions sur de nouveaux projets régionaux.

* Sujet 10: Publications et santé

    Mots-clés: publication, auf, doctorants, santé, coutumiers, compétences, trentaine, agent, dédougou, force\
    Interprétation: Ce sujet traite des publications, de la santé, et de la formation, avec des références à des publications académiques, à la santé publique, et à des discussions sur la formation et les compétences.

    

## Non-negative Matrix Factorization (NMF)

* Sujet 1: Opérations militaires contre des terroristes

    Mots-clés: terroristes, matériel, armées, nigérienne, récupèrent, actualisée, burkinabè, tuent\
    Interprétation: Ce sujet semble traiter des opérations militaires contre des groupes terroristes, avec des références à des confrontations armées, des pertes en vies humaines, et des mises à jour sur les opérations.

* Sujet 2: Accident de la route à Laba

    Mots-clés: accident, Laba, blessés, morts, village, victimes, route, fait\
    Interprétation: Ce sujet concerne un accident de la route survenu à Laba, avec des références à des blessés, des morts, et des détails sur les victimes de l'accident.

* Sujet 3: Projets de construction et développement régional

    Mots-clés: raffinerie, Niger, Dosso, prépare, construire, projet, nouvelle, région, démarrage, ministre\
    Interprétation: Ce sujet aborde des projets de construction et de développement régional, notamment la préparation d'une raffinerie au Niger dans la région de Dosso, avec la participation du gouvernement.

* Sujet 4: Affaires diplomatiques et relations étrangères

    Mots-clés: image, Russie, chef, Normandie, état, étrangères, audience, ministre, affaire, angloaméricains\
    Interprétation: Ce sujet traite des affaires diplomatiques et des relations étrangères, avec des références à des réunions entre dirigeants, des discussions sur l'image internationale, et des affaires impliquant la Russie et les pays anglo-américains.

* Sujet 5: Gestion sanitaire et matériel médical

    Mots-clés: district, sanitaire, Pouytenga, Sankmoney, offre, santé, matériel, agent, excellence, FDS\
    Interprétation: Ce sujet semble concerner la gestion sanitaire et l'approvisionnement en matériel médical, avec des références à des districts sanitaires, des offres de matériel, et des aspects liés à la qualité des services de santé.

* Sujet 6: Problèmes d'électricité et manifestations au Ghana

    Mots-clés: coupures, électricité, Accra, Ghana, manifestation, contre, courant, Lira, NPP, pay\
    Interprétation: Ce sujet évoque des problèmes d'électricité et des manifestations au Ghana, avec des références aux coupures de courant, aux protestations à Accra, et aux réactions politiques.

* Sujet 7: Publications scientifiques et renforcement des compétences

    Mots-clés: publication, AUF, doctorants, compétences, trentaine, scientifiques, renforce, scientifique, Burkina, article\
    Interprétation: Ce sujet traite des publications scientifiques et du renforcement des compétences, notamment des discussions sur les doctorants, les publications académiques, et les initiatives de renforcement des capacités.

* Sujet 8: Formation en couture et coiffure

    Mots-clés: couture, coiffure, apprenantes, promotion, Kourittenga, sortie, elles, fille, formation, élèves\
    Interprétation: Ce sujet semble aborder la formation en couture et coiffure, avec des références à des programmes de promotion, des sorties de formation, et la participation des jeunes filles à des cours de formation.

* Sujet 9: Célébration des cultures et des communautés

    Mots-clés: Bwaba, SNC, culture, fraternité, Peulhs, célèbrent, valeurs, Lanou, Gnoumou, communautés\
    Interprétation: Ce sujet évoque la célébration des cultures et des communautés, avec des références aux valeurs culturelles, aux festivals de la fraternité, et à la participation des communautés locales.

* Sujet 10: Événements historiques et commémorations

    Mots-clés: victoire, African, photo, 79e, anniversaire, russe, initiative, guerre, expose, nazi\
    Interprétation: Ce sujet semble traiter d'événements historiques et de commémorations, avec des références à des victoires militaires, des initiatives commémoratives, et des expositions sur la guerre et le nazisme.

#  BERTopic

* Sujet -1: Santé et Sécurité:

    Mots-clés: santé, terroristes, matériel, agent, district, force, sanitaire, juin, permis, coupures.\
    Interprétation: Ce sujet semble être centré sur les questions de santé et de sécurité, avec des références à des éléments tels que la santé publique, les menaces terroristes, l'équipement médical, les agents de santé, et les incidents comme des coupures de service.

* Sujet 0: Burkina Faso et Informations Générales:

    Mots-clés: Burkina, formation, ministre, couture, président, photo, guerre, information.\
    Interprétation: Ce sujet traite des informations générales sur le Burkina Faso, avec des références à des sujets tels que la formation, les activités gouvernementales, la culture, et la documentation visuelle telle que des photos.

* Sujet 1: Projets et Développement:

    Mots-clés: projet, Niger, raffinerie, tonne, Ioba, région, production, construire, prépare, Dosso.\
    Interprétation: Ce sujet semble concerner les projets de développement, avec des références à des initiatives telles que la construction de raffineries, la production et la préparation de projets dans des régions spécifiques comme le Niger et Ioba.

* Sujet 2: Accidents et Catastrophes:

    Mots-clés: accident, Laba, village, blessés, morts, deux, femmes, nationale, juin, victimes.\
    Interprétation: Ce sujet aborde les accidents et les catastrophes, mettant en lumière des événements tragiques tels que des accidents de la route ou des incidents naturels, avec des références à des lieux spécifiques et au nombre de victimes.

* Sujet 3: SNC et Sensibilisation:

    Mots-clés: SNC, dépisté, comprendre, excité, biennale, Kylian, VIH/SIDA, personne, quel, Bobo.\
    Interprétation: Ce sujet semble être axé sur la sensibilisation et l'éducation, avec des références à des initiatives telles que des programmes de dépistage, la sensibilisation au VIH/SIDA, et des événements éducatifs comme la Biennale, visant à comprendre et à informer les gens sur des questions spécifiques.

5. Comparaison des techniques :\
○ Comparer la cohérence et la pertinence des sujets extraits par chaque
technique.\
○ Discuter des avantages et des inconvénients de chaque méthode de
modélisation de sujets.

### Latent Dirichlet Allocation (LDA)

* Cohérence des sujets :
    Les sujets extraits par LDA sont généralement cohérents, avec des mots-clés souvent sémantiquement liés.\
    Cependant, la cohérence peut varier en fonction du choix des paramètres tels que le nombre de sujets et les hyperparamètres α et β.
* Pertinence des sujets :
    Les sujets sont pertinents pour le corpus, mais leur qualité dépend des paramètres et de la préparation du texte.\
    LDA nécessite souvent un nettoyage préalable du texte pour de meilleurs résultats.
* Avantages :
    Bien établi et largement utilisé pour l'extraction de sujets.\
    Interprétable et facile à mettre en œuvre avec des bibliothèques comme gensim et sklearn.
Inconvénients :
    Sensible au choix des paramètres.\
    Peut être moins performant sur des corpus de grande taille ou hétérogènes.

### Non-negative Matrix Factorization (NMF)

* Cohérence des sujets :
    NMF tend à produire des sujets cohérents, similaires à LDA, mais avec une interprétation mathématique différente basée sur la factorisation des matrices.\
* Pertinence des sujets :
    Les sujets NMF sont souvent pertinents et les mots-clés sont bien groupés en sujets distincts.\
    Peut parfois mieux capturer des sujets de niche ou moins fréquents dans le corpus.
* Avantages :
    Moins sensible aux choix des hyperparamètres comparé à LDA.\
    Peut fournir une meilleure factorisation lorsque les données sont bien adaptées à une décomposition additive.
* Inconvénients :
    Nécessite une matrice TF-IDF ou une matrice de co-occurrence des termes.\
    La sélection du nombre de sujets reste critique pour de bons résultats.

### BERTopic

* Cohérence des sujets :
    BERTopic utilise des modèles de langage pré-entraînés pour générer des représentations vectorielles, ce qui peut conduire à des sujets très cohérents et spécifiques.\
    Capable de capturer des relations complexes entre les mots grâce aux représentations contextuelles.
* Pertinence des sujets :
    Les sujets extraits sont souvent très pertinents pour le corpus, avec des thèmes bien définis.\
    Les modèles de langage pré-entraînés comme BERT permettent une compréhension plus profonde du contexte.
* Avantages :
    Ne nécessite pas de nombre de sujets prédéfini.\
    Efficace pour l'analyse de grands corpus de texte.\
    Peut intégrer des représentations vectorielles contextuelles pour des sujets plus riches.
* Inconvénients :
    Plus complexe à mettre en œuvre que LDA ou NMF.\
    Sensible au choix du modèle de langage pré-entraîné et des paramètres.\
    Interprétation des sujets peut être plus difficile en raison de l'abstraction des représentations vectorielles.\

### Conclusion

En résumé, chaque méthode a ses propres avantages et inconvénients. Le choix de la méthode dépend des objectifs spécifiques de l'analyse et des caractéristiques du corpus de texte :

    LDA est bien adapté pour une analyse exploratoire simple et interprétable, mais peut être limité par la nécessité de choisir des paramètres appropriés.
    NMF offre une alternative intéressante à LDA avec une meilleure performance dans certains cas, particulièrement pour des sujets de niche.
    BERTopic est puissant pour des analyses complexes et contextuelles, mais requiert une mise en œuvre plus sophistiquée et peut être difficile à interpréter.

Pour des analyses rapides et interprétables, LDA ou NMF sont souvent suffisants. Pour des analyses plus profondes et contextuelles, BERTopic peut offrir des avantages significatifs malgré sa complexité.

In [7]:
pip install streamlit sklearn  matplotlib plotly gensim pandas



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag